In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
!pip install demoji
import demoji
demoji.download_codes()
nltk.download('stopwords') 
nltk.download('wordnet')

... OK (Got response in 0.14 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
df = pd.read_json('/content/drive/MyDrive/sarcasm/reddit/sarcasm_detection_shared_task_reddit_training.jsonl',lines=True)
df1 = pd.read_json('/content/drive/MyDrive/sarcasm/reddit/sarcasm_detection_shared_task_reddit_testing.jsonl',lines=True)
#df1['response'] = df1['response'].apply(preprocess_text)

In [4]:
df.head()

,label,response,context
0,SARCASM,"Yeah I mean there's only one gender anyways, w...",[LPT: If you're worried about hurting someone'...
1,SARCASM,"Sounds like you don't like science, you theist...",[Promotional images for some guy's Facebook pa...
2,SARCASM,"Ofc play them in try mode, Blizzard were so ge...",[My friends won't play Dota2; I won't play LoL...
3,SARCASM,"I don't understand, Reddit told me that Hillar...",[Poll: Convention boosts Clinton to 11-point l...
4,SARCASM,"yeh, they're the reigning triple premiers, why...",[Wayne Ludbey: Jordan Lewis has the ultimate c...


In [5]:
for i in range(10):
  print(df1['context'][i])
  print(df1['response'][i])
  print(df1['label'][i])

['Dear Trump: the bill is now on your desk. Veto the shit out of it to protect the privacy of people YOU work for. Grow some and stand up to the swamp you supposedly were going to drain. If you dont, we WILL fight back', "If you think Trump gives matters like this anything more than a moment's consideration and a snap decision based on who he's talking to, you haven't been paying attention.", 'Mr. President, here is a bill that was unanimously supported by Republicans and unanimously opposed by Democrats. "Where do I sign?" The bill concerns Internet companies collecti... "Where do I sign?"', 'Oh and by the way this is repealing a regulation that Obama wrote, so do with it as you will', 'Calling it a decision is pretty generous.', '*I was elected to LEAD not to READ*']
I was elected to golf not to uh, got nothing.
NOT_SARCASM
['Thailand cave rescue: All 12 boys, coach freed, latest updates', 'God damn Thai Navy SEALs and the crew that planned it out are really something else. Cutting a

In [ ]:

df['context_resp']=df['label']
for i in range(len(df['label'])):
  df['context_resp'][i]=''.join(df['context'][i]) + df['response'][i]

print(df['context'][12])
print(df['response'][12])
print(df['context_resp'][12])

['If Trump Is Innocent, Why Is He Acting So Guilty?', "If Trump is so innocent, then why is literally everyone he's surrounding himself with linked back to Russia?"]
Um sorry I don't listen to paid shills who spout out obviously fake news
If Trump Is Innocent, Why Is He Acting So Guilty?If Trump is so innocent, then why is literally everyone he's surrounding himself with linked back to Russia?Um sorry I don't listen to paid shills who spout out obviously fake news


In [ ]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation(https://www.nltk.org/_modules/nltk/tokenize/regexp.html)
    text = re.sub(r"@USER","", text)
    text = re.sub(r'<URL>','',text)
    text = demoji.replace_with_desc(text)
    text = re.sub(r':','',text)
    # tokeniser = RegexpTokenizer(r'\s+', gaps=True)
    # tokeniser = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)

    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    
    # stemming
    ps=PorterStemmer()
    keywords=[ps.stem(x) for x in keywords]
    return keywords

def preprocess_text2(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation (optional)
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [ ]:
for i in range(5):
  print(df['response'][i])
  print(preprocess_text(df['response'][i]))
  print(preprocess_text2(df['response'][i]))
  print()
  print(df['context_resp'][i])
  print(preprocess_text(df['context_resp'][i]))
  print(preprocess_text2(df['context_resp'][i]))
  print()

Yeah I mean there's only one gender anyways, women are objects
['yeah', 'mean', 'one', 'gender', 'anyway', 'women', 'object']
['yeah', 'mean', "there'", 'one', 'gender', 'anyway', 'women', 'object']

LPT: If you're worried about hurting someone's feelings by using the incorrect gender pronoun, play it safe and spare them any unnecessary pain by using the neutral word "it"When gender is unknown he/him is default.Yeah I mean there's only one gender anyways, women are objects
['lpt', 'worri', 'hurt', 'someon', 'feel', 'use', 'incorrect', 'gender', 'pronoun', 'play', 'safe', 'spare', 'unnecessari', 'pain', 'use', 'neutral', 'word', 'gender', 'unknown', 'default', 'yeah', 'mean', 'one', 'gender', 'anyway', 'women', 'object']
['lpt', 'worri', 'hurt', "someone'", 'feel', 'use', 'incorrect', 'gender', 'pronoun', 'play', 'safe', 'spare', 'unnecessari', 'pain', 'use', 'neutral', 'word', 'gender', 'unknown', 'default.yeah', 'mean', "there'", 'one', 'gender', 'anyway', 'women', 'object']

Sounds l

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
print(df['label'].value_counts())
print(df1['label'].value_counts())

SARCASM        2200
NOT_SARCASM    2200
Name: label, dtype: int64
SARCASM        900
NOT_SARCASM    900
Name: label, dtype: int64


In [ ]:
from sklearn.utils import shuffle
X_train, Y_train = shuffle(df['response'],df['label'],random_state=123)
X_test, Y_test = df1['response'],df1['label']

**TF-idf**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to feature matrix
X_train_tfidf = vectoriser.fit_transform(X_train)
print(X_train_tfidf.shape)

X_test_tfidf = vectoriser.transform(X_test)
print(X_test_tfidf.shape)

(4400, 6784)
(1800, 6784)


In [ ]:
#all ml models
# !pip install nbsvm-sklearn
# from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# first, initialize the classificators
# nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24)
xboost= XGBClassifier(random_state=24)


# now, create a list with the objects 
models= [tree, forest, knn, svm, xboost]
#models=[nbsvm]
for model in models:
    model.fit(X_train_tfidf, Y_train) # fit the model
    y_pred= model.predict(X_test_tfidf) # then predict on the test set
    accuracy= accuracy_score(Y_test, y_pred) # this gives us how often the algorithm predicted correctly
    clf_report= classification_report(Y_test, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
    print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    print(clf_report)
    print("\n")

The accuracy of model DecisionTreeClassifier is 0.57
              precision    recall  f1-score   support

 NOT_SARCASM       0.56      0.60      0.58       900
     SARCASM       0.57      0.53      0.55       900

    accuracy                           0.57      1800
   macro avg       0.57      0.57      0.57      1800
weighted avg       0.57      0.57      0.57      1800



The accuracy of model RandomForestClassifier is 0.57
              precision    recall  f1-score   support

 NOT_SARCASM       0.56      0.66      0.61       900
     SARCASM       0.59      0.49      0.53       900

    accuracy                           0.57      1800
   macro avg       0.58      0.57      0.57      1800
weighted avg       0.58      0.57      0.57      1800



The accuracy of model KNeighborsClassifier is 0.52
              precision    recall  f1-score   support

 NOT_SARCASM       0.52      0.49      0.50       900
     SARCASM       0.52      0.56      0.54       900

    accuracy         

**Doc2Vec**

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
tagged_data = [TaggedDocument(words=preprocess_text(_d), tags=[str(i)]) for i, _d in enumerate(X_train)]
tagged_data[:5]

[TaggedDocument(words=['gay', 'marriag', 'legal', '2005', 'take', 'guy', 'ten', 'year', 'mayb', 'eventu', 'least', 'two', 'year'], tags=['0']),
 TaggedDocument(words=['find', 'lack', 'faith', 'reddit', 'disturb'], tags=['1']),
 TaggedDocument(words=['encourag', 'continu', 'men'], tags=['2']),
 TaggedDocument(words=['alot', 'differ', 'roll', 'back', 'attidu', 'like', '2x', 'scale', '2x', 'detail', 'throw', 'photoshop', 'scale'], tags=['3']),
 TaggedDocument(words=['kid', 'french', 'map', 'know', 'lot', 'corsican', 'would', 'love'], tags=['4'])]

In [ ]:
max_epochs =100
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)


for epoch in tqdm(range(max_epochs)):
    # print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=1)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    # model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")


Model Saved


In [ ]:
model= Doc2Vec.load("d2v.model")

X_train_d2v=[model.infer_vector(preprocess_text(X_train[i])) for i in range(len(X_train))]
X_train_d2v=np.array(X_train_d2v)

X_test_d2v=[model.infer_vector(preprocess_text(X_test[i])) for i in range(len(X_test))]
X_test_d2v=np.array(X_test_d2v)

In [ ]:
# X_train1, X_test1, Y_train1, Y_test1=train_test_split(X_train_d2v,Y_train,random_state=42)
X_train1, X_test1, Y_train1, Y_test1=X_train_d2v, X_test_d2v, Y_train, Y_test

In [ ]:
#all ml models
# !pip install nbsvm-sklearn
#from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

# first, initialize the classificators
#nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24)
xboost= XGBClassifier(random_state=24)

# X_test_tfidf = vectoriser.transform(X_test)
# print(X_test_tfidf.shape)
# now, create a list with the objects 
models= [tree, forest, knn, svm, xboost]
#models=[nbsvm]
for model in models:
    model.fit(X_train1, Y_train1) # fit the model
    y_pred= model.predict(X_test1) # then predict on the test set
    accuracy= accuracy_score(Y_test1, y_pred) # this gives us how often the algorithm predicted correctly
    clf_report= classification_report(Y_test1, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
    print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    print(clf_report)
    print("\n")

The accuracy of model DecisionTreeClassifier is 0.49
              precision    recall  f1-score   support

 NOT_SARCASM       0.49      0.49      0.49       900
     SARCASM       0.49      0.48      0.48       900

    accuracy                           0.49      1800
   macro avg       0.49      0.49      0.49      1800
weighted avg       0.49      0.49      0.49      1800



The accuracy of model RandomForestClassifier is 0.50
              precision    recall  f1-score   support

 NOT_SARCASM       0.50      0.53      0.51       900
     SARCASM       0.50      0.47      0.48       900

    accuracy                           0.50      1800
   macro avg       0.50      0.50      0.50      1800
weighted avg       0.50      0.50      0.50      1800



The accuracy of model KNeighborsClassifier is 0.48
              precision    recall  f1-score   support

 NOT_SARCASM       0.48      0.47      0.48       900
     SARCASM       0.48      0.49      0.49       900

    accuracy         


**Universal Sentence Encoder**

In [ ]:
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub

Requirement already up-to-date: tensorflow-gpu in /usr/local/lib/python3.7/dist-packages (2.5.0)


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
X_train_use = model(list(X_train.values))
X_test_use  = model(list(X_test.values))
print(X_train_use.shape)
print(X_test_use.shape)

(4400, 512)
(1800, 512)


In [ ]:
X_train3, X_test3, Y_train3, Y_test3=X_train_use, X_test_use, Y_train, Y_test

In [ ]:
#all ml models
# !pip install nbsvm-sklearn
#from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

# first, initialize the classificators
#nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24)
xboost= XGBClassifier(random_state=24)
gnb = GaussianNB()

# now, create a list with the objects 
models= [tree, forest, knn, svm, xboost, gnb]
#models=[nbsvm]
for model in models:
    model.fit(X_train3, Y_train3) # fit the model
    y_pred= model.predict(np.array(X_test3)) # then predict on the test set
    accuracy= accuracy_score(Y_test3, y_pred) # this gives us how often the algorithm predicted correctly
    clf_report= classification_report(Y_test3, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
    print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    print(clf_report)
    print("\n")

The accuracy of model DecisionTreeClassifier is 0.54
              precision    recall  f1-score   support

 NOT_SARCASM       0.54      0.54      0.54       900
     SARCASM       0.54      0.53      0.53       900

    accuracy                           0.54      1800
   macro avg       0.54      0.54      0.54      1800
weighted avg       0.54      0.54      0.54      1800



The accuracy of model RandomForestClassifier is 0.64
              precision    recall  f1-score   support

 NOT_SARCASM       0.62      0.71      0.66       900
     SARCASM       0.66      0.56      0.60       900

    accuracy                           0.64      1800
   macro avg       0.64      0.64      0.63      1800
weighted avg       0.64      0.64      0.63      1800



The accuracy of model KNeighborsClassifier is 0.58
              precision    recall  f1-score   support

 NOT_SARCASM       0.59      0.54      0.57       900
     SARCASM       0.58      0.62      0.60       900

    accuracy         

##Using Sentence Transformer embedding
https://github.com/UKPLab/sentence-transformers

In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 2.3MB 8.4MB/s 
     |████████████████████████████████| 1.2MB 37.3MB/s 
     |████████████████████████████████| 3.3MB 41.0MB/s 
     |████████████████████████████████| 901kB 19.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=9aef30d70115d313a2a8c15009246c8fff932029ac2a94c8ad505c1b96655395
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
# sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
sbert_model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [ ]:
# sgd_clf = SGDClassifier(random_state=123)
from tqdm.auto import tqdm
import numpy as np
query = "I had pizza and pasta"
# sentence_embeddings = sbert_model.encode(query) 
sentence_embeddings = [sbert_model.encode(i.lower()) for i in tqdm(X_train)]

In [ ]:
query = "I had pizza and pasta"
# sentence_embeddings = sbert_model.encode(query) 
test_embeddings = [sbert_model.encode(i.lower()) for i in tqdm(X_test)]

In [ ]:
sentence_embeddings=np.array(sentence_embeddings)
test_embeddings=np.array(test_embeddings)

In [ ]:
# X_train2, X_test2, Y_train2, Y_test2=train_test_split(sentence_embeddings,Y_train,random_state=42)
X_train2, X_test2, Y_train2, Y_test2=sentence_embeddings, test_embeddings, Y_train, Y_test

In [ ]:
#all ml models
# !pip install nbsvm-sklearn
#from nbsvm import NBSVMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

# first, initialize the classificators
# nbsvm = NBSVMClassifier()
tree= DecisionTreeClassifier(random_state=24) # using the random state for reproducibility
forest= RandomForestClassifier(random_state=24)
knn= KNeighborsClassifier()
svm= SVC(random_state=24)
xboost= XGBClassifier(random_state=24)
gnb = GaussianNB()
# y_pred = gnb.fit(X_train, y_train).predict(X_test)

# X_test_tfidf = vectoriser.transform(X_test)
# print(X_test_tfidf.shape)
# now, create a list with the objects 
models= [tree, forest, knn, svm, xboost, gnb]
#models=[nbsvm]
for model in models:
    model.fit(X_train2, Y_train2) # fit the model
    y_pred= model.predict(X_test2) # then predict on the test set
    accuracy= accuracy_score(Y_test2, y_pred) # this gives us how often the algorithm predicted correctly
    clf_report= classification_report(Y_test2, y_pred) # with the report, we have a bigger picture, with precision and recall for each class
    print(f"The accuracy of model {type(model).__name__} is {accuracy:.2f}")
    print(clf_report)
    print("\n")

The accuracy of model DecisionTreeClassifier is 0.52
              precision    recall  f1-score   support

 NOT_SARCASM       0.52      0.54      0.53       900
     SARCASM       0.52      0.50      0.51       900

    accuracy                           0.52      1800
   macro avg       0.52      0.52      0.52      1800
weighted avg       0.52      0.52      0.52      1800



The accuracy of model RandomForestClassifier is 0.60
              precision    recall  f1-score   support

 NOT_SARCASM       0.59      0.66      0.62       900
     SARCASM       0.62      0.54      0.58       900

    accuracy                           0.60      1800
   macro avg       0.60      0.60      0.60      1800
weighted avg       0.60      0.60      0.60      1800



The accuracy of model KNeighborsClassifier is 0.54
              precision    recall  f1-score   support

 NOT_SARCASM       0.55      0.40      0.46       900
     SARCASM       0.53      0.67      0.59       900

    accuracy         

##Bag-of-words